In [54]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

import re

from datetime import datetime

In [2]:
driver = webdriver.Firefox()


In [3]:
urls = [f"https://www.oddsportal.com/basketball/usa/nba-{x}-{x+1}/results/" for x in range(2008, 2025)]
urls_shortened = [f"/basketball/usa/nba-{x}-{x+1}/" for x in range(2008, 2025)]

In [4]:
total_game_urls = []

In [8]:
for url, url_shortened in zip(urls[-4:], urls_shortened[-4:]):
    driver.get(url)

    WebDriverWait(driver, 10).until(
            lambda driver: driver.execute_script("return document.readyState") == "complete"
    )

    try:

        wait = WebDriverWait(driver, 5)
        link = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler")))

        link.click()
    
    except: 

        print("timeout")

    parsed = BeautifulSoup(driver.page_source)

    pagination = parsed.find("div", attrs={"class": "pagination"})

    total_pages = int(pagination.find_all("a")[-2]["data-number"])

    games = parsed.find_all("div", attrs={"class" : "eventRow"})

    total_game_urls.extend([x.find("a", attrs={"href": re.compile(url_shortened)})["href"] for x in games][1:])

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    time.sleep(2)

    page_number = 0

    while page_number < total_pages:
        wait = WebDriverWait(driver, 10)
        link = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "Next")))

        assert link.is_displayed()

        link.click()

        WebDriverWait(driver, 10).until(
            lambda driver: driver.execute_script("return document.readyState") == "complete"
        )

        time.sleep(2)

        parsed = BeautifulSoup(driver.page_source)

        games = parsed.find_all("div", attrs={"class" : "eventRow"})

        total_game_urls.extend([x.find("a", attrs={"href": re.compile(url_shortened)})["href"] for x in games][1:])

        page_number = int(driver.current_url.split("/")[-2])

timeout
timeout
timeout
timeout


AttributeError: 'NoneType' object has no attribute 'find_all'

In [11]:
with open("total_game_urls", "w") as f:
    f.writelines("\n".join(list(set(total_game_urls))))



In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

import re

import ndjson

from tqdm import tqdm

from datetime import datetime

In [2]:
with open("total_game_urls", "r") as f:
    scraped_urls = [x.strip() for x in f.readlines()]

In [3]:
driver = webdriver.Firefox()

# odds_total = []

with open("data/odds/odds4.ndjson", "w") as f:

    writer = ndjson.writer(f, ensure_ascii=False)

    for scraped_url in tqdm(scraped_urls[7471+854+3299:]):
        driver.get("https://www.oddsportal.com" + scraped_url)

        WebDriverWait(driver, 10).until(
            lambda driver: driver.execute_script("return document.readyState") == "complete"
        )

        parsed = BeautifulSoup(driver.page_source)

        odds_data = [[y.text for y in x.parent.find_all("p")] for x in parsed.find_all("div", attrs={"provider-name":'0'})]

        date = parsed.find("div", attrs={"class": "bg-event-start-time"}).parent.text

        # date_parsed = datetime.strptime(date, '%A,%d %b %Y,%H:%M')

        team1, team2 = [x.text for x in parsed.find_all("span", attrs={"class": "truncate"})]

        team1_score, team2_score = [x.text for x in parsed.find_all("div", attrs={"class": re.compile("text-gray-dark flex")})[:2]]

        data = {
            "Home" : team1,
            "Away" : team2,
            "Home Score" : team1_score,
            "Away Score" : team2_score,
            "Date": date,
            "Bookmakers" : [x[0] for x in odds_data],
            "Home Odds" : [x[1] for x in odds_data],
            "Away Odds" : [x[2] for x in odds_data],
        }

        writer.writerow(data)

100%|██████████| 5928/5928 [2:43:20<00:00,  1.65s/it]  
